In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-25 20:33:50,767 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-25 20:33:50,769 - settings - Setting database.user to celiib
INFO - 2021-05-25 20:33:50,769 - settings - Setting database.password to newceliipass
INFO - 2021-05-25 20:33:50,774 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-25 20:33:50,774 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-25 20:33:50,788 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-25 20:33:51,285 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-25 20:33:51,289 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-25 20:33:51,290 - settings - Setting database.user to celiib
INFO - 2021-05-25 20:33:51,290 - settings - Setting database.password to newceliipass
INFO - 2021-05-25 20:33:51,291 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-25 20:33:51,291 - settings - Setting database.user to celiib
INFO - 2021-05-25 20:33:51,292 - settings - Setting database.password to newceliipass
INFO - 2021-05-25 20:33:51,294 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-25 20:33:51,830 - settings - Setting enable_python_native_blobs to True


In [3]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-05-25 20:33:51,978 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-25 20:33:51,980 - settings - Setting database.user to celiib
INFO - 2021-05-25 20:33:51,981 - settings - Setting database.password to newceliipass
INFO - 2021-05-25 20:33:51,984 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-25 20:33:52,588 - settings - Setting enable_python_native_blobs to True


# Getting the Segment Ids to Download

In [4]:
minnie.AutoProofreadValidationSegmentMap4()

ver materialization version,nucleus_id nucleus id,segment_id the segment id in the proofread version,old_ver materialization version,old_segment_id the segment id in the proofread version--,finished_dendrite whether axon was finished cleaning,finished_axon whether axon was finished cleaning,extension_type,n_somas,n_soma_mergers,n_glia,large_splits,notes
88.00,189149,864691135855890478,0.08,864691135585319036,True,True,full,1,0,0,0,
88.00,223037,864691136227167569,0.08,864691135538120050,True,True,full,2,0,0,0,
88.00,225498,864691135753932237,0.08,864691135564655959,True,True,full,1,0,0,0,
88.00,230236,864691135609687047,0.08,864691135748568361,True,True,full,1,0,0,0,
88.00,236197,864691136903144370,0.08,864691135758569550,True,True,full,2,0,2,0,
88.00,258307,864691135293126156,0.08,864691136194042326,True,True,full,1,0,0,0,
88.00,259167,864691134988722810,0.08,864691135700248187,True,True,full,2,0,0,0,
88.00,262773,864691136023889209,0.08,864691135233108569,True,True,full,1,0,0,0,
88.00,264870,864691136951664863,0.08,864691135740225387,True,True,full,1,0,0,0,
88.00,269247,864691135584074360,0.08,864691136105498585,True,True,full,1,0,0,0,


In [11]:
missing_ids = (minnie.AutoProofreadValidationSegmentMap4() - minnie.Mesh()).fetch("segment_id")
missing_ids

array([864691136227167569, 864691135753932237, 864691135609687047,
       864691134988722810, 864691135584074360, 864691135974639471,
       864691136011067043, 864691136084313196, 864691135508871945,
       864691136309871706, 864691136020550392, 864691135873703054,
       864691136136778621, 864691136084302956, 864691135013115542,
       864691136118618136, 864691136004983754, 864691135761858870,
       864691135724453547, 864691135360256583, 864691136237620540,
       864691135194621738, 864691136011081123, 864691136194355176,
       864691136175223814, 864691135654275266, 864691135467798412,
       864691135567953132, 864691135081943287, 864691136329122794,
       864691135593719339, 864691135926404564, 864691135428676144,
       864691136237644604, 864691135155702116, 864691136118618392,
       864691136451224831, 864691136136766589, 864691135358995288,
       864691135501703746, 864691136105661657, 864691135382687962,
       864691135754839762, 864691135785571780, 864691135445710

In [ ]:
du.download_and_insert_allen_meshes(missing_ids,
                                   n_threads=4)

In [ ]:
minnie.AutoProofreadValidationSegment4()

# Still Missing Segment Ids

In [9]:
minnie.AutoProofreadValidationSegmentMap4() & "segment_id=864691136105661657"

ver materialization version,nucleus_id nucleus id,segment_id the segment id in the proofread version,old_ver materialization version,old_segment_id the segment id in the proofread version--,finished_dendrite whether axon was finished cleaning,finished_axon whether axon was finished cleaning,extension_type,n_somas,n_soma_mergers,n_glia,large_splits,notes
88.00,582294,864691136105661657,0.08,864691136740414556,True,True,partial,1,0,0,0,


In [6]:
du.current_validation_segment_id_restriction & dict(segment_id=864691136105661657)

segment_id the segment id in the proofread version--


In [5]:
key_source = du.current_validation_segment_id_restriction - (minnie.Decimation() & dict(decimation_ratio=0.25)).proj()
seg_ids_still_miss = key_source.fetch("segment_id")
seg_ids_still_miss

array([], dtype=int64)

In [ ]:
du.download_and_insert_allen_meshes(seg_ids_still_miss,
                                   n_threads=5)